In [3]:
import django
import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mysite.settings')
django.setup()

In [4]:
from fbpost.models import *
from datetime import datetime

In [5]:
def dateformat(postedtime):
    return postedtime.strftime("%Y-%m-%d %H:%M:%S.%f")

In [6]:
def get_comments(obj):
    comment_dictionary = {}
    comment_dictionary["comment_id"] = obj.id
    comment_dictionary["commenter"] = get_user(obj)
    comment_dictionary["commented_at"] = dateformat(obj.comment_at)
    comment_dictionary["comment_content"] = obj.comment_content
    comments_reactions = get_reactions(obj)
    comment_dictionary['reactions'] = {"count":len(comments_reactions),"type":list(set(comments_reactions))}
    return comment_dictionary

In [7]:

def get_user(obj):
    object_dictionary = {}
    object_dictionary["name"] = obj.user.username
    object_dictionary["user_id"] = obj.user.id
    object_dictionary["profile_pic_url"] = obj.user.pic_url
    return object_dictionary

In [8]:
def create_post(user_id,post_content):
    try:
        u=User.objects.get(id=user_id)
    except User.DoesNotExist:
        print("User Not Found")
        return
    p1=Post.objects.create(post_datetime=dateformat(datetime.now()),post_content=post_content,user=u)
    return p1.id

In [9]:
def get_reactions(obj):
    list_of_reactions=[]
    for reaction_object in obj.reaction.all():
        list_of_reactions.append(reaction_object.react_type)
    return list_of_reactions

# def get_post(post_id):
    try:
        post=Post.objects.get(id=post_id)
    except User.DoesNotExist:
        print("Post Not Found")
        return
    d={}
    d['post_id']=post_id
    d['posted_by']=get_user(post)
    d['posted_at']=str(post.post_datetime)[:-6]
    d['post_content']=post.post_content
    l=get_reactions(post)
    d['reactions']={"count":len(l),"type":list(set(l))}
    return d
    
    
        

In [11]:
def get_user_posts(user_id):
    try:
        User.objects.get(id=user_id)
    except User.DoesNotExist:
        print("User Not Found")
    l=[]
    user=User.objects.get(id=user_id)
    for i in user.posts.all():
        l.append(get_post(i.id))
    return l

In [12]:
def delete_post(post_id):
    try:
        post = Post.objects.get(id=post_id)
    except Post.DoesNotExist:
        print("Post Not Found")
        return
    post.delete()

In [13]:
def react_to_post(user_id,post_id,reaction_type):
    try:
        User.objects.get(id=user_id)
    except User.DoesNotExist:
        print("Invalid User")
        return
    try:
        Post.objects.get(id=post_id)
    except Post.DoesNotExist:
        print("Invalid Post")
        return
    if(reaction_type not in ["HAHA","LIKE","SAD","WOW","LOVE"]):
        print("Reaction doesnot exist")
        return 
        
    try:
        reaction=Reactions.objects.get(user_id=user_id,post_id=post_id)
    except Reactions.DoesNotExist:
        print("No Reaction Found")
        r=Reactions.objects.create(react_type=reaction_type,post=Post.objects.get(id=post_id),user=User.objects.get(id=user_id))
        return
    if(reaction_type==reaction.react_type):
        reaction.delete()
    else:
        reaction.react_type=reaction_type
        reaction.save()
        

        
    

In [14]:
def get_posts_reacted_by_user(user_id):
    try:
        u=User.objects.get(id=user_id)
    except User.DoesNotExist:
        print("User Not Found")
        return
    l=[]
    reaction=Reactions.objects.filter(user=u)
    for i in reaction:
        p=i.post
        l.append(p.id)
    return l
        

In [15]:
def get_reactions_to_post(post_id):
    try:
        p=Post.objects.get(id=post_id)
    except Post.DoesNotExist:
        print("Post Not Found")
        return
    l=[]
    reaction_query_set=Reactions.objects.filter(post=p)
#     print(reaction)
    for reaction_object in reaction_query_set:
        d=get_user(reaction_object)
        d['reaction']=reaction_object.react_type
        l.append(d)
    return l
    

In [16]:
def get_reaction_metrics(post_id):
    try:
        p=Post.objects.get(id=post_id)
    except Post.DoesNotExist:
        print("Post Not Found")
        return
    d={}
    reaction=Reactions.objects.filter(post=p)
    for i in reaction:
        if i.react_type not in d.keys():
            d[i.react_type]=1
        else:
            d[i.react_type]+=1
    return d
            
    

In [17]:
def get_posts_with_positive_reactions():
    pos=['LIKE','LOVE','HAHA','WOW']
    neg=['ANGRY','SAD']
    l=[]
    post=Post.objects.all()
    for i in post:
        r=i.reaction.all()
        p,n=0,0
        for j in r:
            if j.react_type in pos:
                p+=1
            elif j.react_type in neg:
                n+=1
        if(p>n):
            l.append(i.id)
    return l
    

# FBV2

In [18]:
def add_comment(post_id,user_id,comment_text):
    try:
        u=User.objects.get(id=user_id)
    except User.DoesNotExist:
        print("Invalid User")
        return
    try:
        p=Post.objects.get(id=post_id)
    except Post.DoesNotExist:
        print("Invalid Post")
        return
    c=Comment.objects.create(user=u,post=p,comment_at=str(datetime.now()),comment_content=comment_text)
    return c.id
    

def get_post(post_id):
    try:
        post=Post.objects.get(id=post_id)
    except User.DoesNotExist:
        print("Post Not Found")
        return

    post_dictionary = {}
    post_dictionary['post_id'] = post_id
    post_dictionary['posted_by'] = {
        "name": post.user.username,
        "user_id":post.user.id,
        "profile_pic_url":post.user.pic_url
    }
    post_dictionary['posted_at'] = str(post.post_datetime)[:-6]
    post_dictionary['post_content'] = post.post_content
    
    post_reactions = []
    for reaction_obj in post.reaction.all():
        post_reactions.append(reaction_obj.react_type)
    post_dictionary['reactions']={
        "count":len(post_reactions),
        "type":list(set(post_reactions))
    }
    
    comment_queryset=post.comments.all()
    list_of_comments_dictionary = []
    for comment in comment_queryset:
        comment_dictionary = {}
        comment_dictionary["comment_id"]=comment.id
        comment_dictionary["commenter"]= {
            "name":comment.user.username,
            "user_id":comment.user.id,
            "profile_pic_url":comment.user.pic_url
        }
        comment_dictionary["commented_at"]=str(comment.comment_at)
        comment_dictionary["comment_content"]=comment.comment_content
        comments_reactions = []
        for reaction_obj in comment.reaction.all():
            comments_reactions.append(reaction_obj.react_type)
        comment_dictionary['commentreactions']={
            "count":len(comments_reactions),
            "type":list(set(comments_reactions))
        }
        list_of_comments_dictionary.append(comment_dictionary)
    print(post_dictionary)
    print('\n\n')
    print(list_of_comments_dictionary)
    # 
    post_dictionary["comments"]=list_of_comments_dictionary
    post_dictionary["comments_count"]=len(list_of_comments_dictionary)
    return post_dictionary
    
    
        

In [ ]:
def react_to_comment(user_id,comment_id,reaction_type):
    try:
        user=User.objects.get(id=user_id)
    except User.DoesNotExist:
        print("Invalid User")
        return
    try:
        comment=Comment.objects.get(id=comment_id)
    except Comment.DoesNotExist:
        print("Invalid Comment")
        return
    if(reaction_type not in ["HAHA","LIKE","SAD","WOW","LOVE"]):
        print("Reaction doesnot exist")
        return 
        
    try:
        reaction=Reactions.objects.get(user_id=user_id,comment_id=comment_id)
    except Reactions.DoesNotExist:
        print("No Reaction Found")
        r=Reactions.objects.create(react_type=reaction_type,user=user,comment=comment)
        return
    if(reaction_type==reaction.react_type):
        reaction.delete()
    else:
        reaction.react_type=reaction_type
        reaction.save()
        

# FBv3

In [ ]:
def reply_to_comment(comment_id,user_id,reply_text):
    try:
        user=User.objects.get(id=user_id)
    except User.DoesNotExist:
        print("Invalid User")
        return
    try:
        comment=Comment.objects.get(id=comment_id)
    except Comment.DoesNotExist:
        print("Invalid Comment")
        return
    
    reply=Comment.objects.create(parent_comment=comment,user=user,comment_at=str(datetime.now()),comment_content=reply_text)
    return reply.id

In [20]:
def get_post(post_id):
    try:
        post = Post.objects.get(id=post_id)
    except User.DoesNotExist:
        print("Post Not Found")
        return

    post_dictionary = {}
    post_dictionary['post_id'] = post_id
    post_dictionary['posted_by'] = get_user(post)
    post_dictionary['posted_at'] = dateformat(post.post_datetime)
    post_dictionary['post_content'] = post.post_content

    post_reactions = get_reactions(post)
    post_dictionary['reactions'] = {
        "count": len(post_reactions),
        "type": list(set(post_reactions))
    }

    comment_queryset = post.comments.all()
    list_of_comments_dictionary = []
    for comment in comment_queryset:
        comment_dictionary = get_comments(comment)
        list_of_reply_dictionary = []
        reply_query_set = comment.reply.all()
        for reply in reply_query_set:
            reply_dictionary = get_comments(reply)
            list_of_reply_dictionary.append(reply_dictionary)
        comment_dictionary["replies_count"] = len(list_of_reply_dictionary)
        comment_dictionary["replies"] = list_of_reply_dictionary
        list_of_comments_dictionary.append(comment_dictionary)
    post_dictionary["comments"] = list_of_comments_dictionary
    post_dictionary["comments_count"] = len(list_of_comments_dictionary)
    return post_dictionary

In [34]:
def get_replies_for_comment(comment_id):
    try:
        comment=Comment.objects.get(id=comment_id)
    except Comment.DoesNotExist:
        print("Invalid Comment")
        return
    list_of_reply_dictionary=[]
    reply_query_set=comment.reply.all()
    for reply in reply_query_set:
        list_of_reply_dictionary=get_comments(reply)
        list_of_reply_dictionary.pop('reactions', None)
    return list_of_reply_dictionary
    

In [36]:
get_replies_for_comment(1)

{'comment_id': 5,
 'commenter': {'name': 'narendra',
  'user_id': 1,
  'profile_pic_url': 'https://dummy.url.com/pic3.png'},
 'commented_at': '2019-07-08 06:38:32.701713',
 'comment_content': 'What man'}

In [25]:
Comment.objects.all()

<QuerySet [<Comment: Comment object (1)>, <Comment: Comment object (2)>, <Comment: Comment object (3)>, <Comment: Comment object (4)>, <Comment: Comment object (5)>]>